In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import random
from PIL import ImageDraw
from matplotlib import pyplot as plt
from pathlib import Path

IMG_SIZE = 224

# 模型載入
print("開始載入 VGG19 模型...")
vgg_model = tf.keras.models.load_model("./model/vgg19_transfer_model.h5")
print("已載入 VGG19 模型")

print("開始載入 Cat Generator 模型...")
cat_gen = tf.keras.models.load_model("./model/cat_pix2pix_generator_model.h5")
print("已載入 Cat Generator 模型")

print("開始載入 Dog Generator 模型...")
dog_gen = tf.keras.models.load_model("./model/dog_pix2pix_generator_model.h5")
print("已載入 Dog Generator 模型")

# 預測與生成主功能
def predict_and_generate(image):
    
    if isinstance(image, dict) and "layers" in image:
        # 來自 Sketchpad，取最後一層
        last_layer = image["layers"][-1]
        image = Image.fromarray(last_layer).convert("RGB")
    elif isinstance(image, np.ndarray):
        image = Image.fromarray(image).convert("RGB")
    elif isinstance(image, Image.Image):
        image = image.convert("RGB")
    else:
        return "輸入格式錯誤", None

    # 儲存原圖做比對
    image.save("./output/input_saved.png")

    # 處理為 VGG19 輸入
    image_rgb = image.convert("RGB").resize((IMG_SIZE, IMG_SIZE))
    img_array = np.array(image_rgb).astype(np.float32)
    input_tensor = img_array.reshape((1, IMG_SIZE, IMG_SIZE, 3))

    # 分類
    preds = vgg_model.predict(input_tensor)[0]
    label = np.argmax(preds)
    confidence = preds[label] * 100

    if label == 0:
        label_text = "cat"
        generator = cat_gen
    elif label == 1:
        label_text = "dog"
        generator = dog_gen
    else:
        return "other (不生成)", image

    # 生成圖（256x256）
    image_resized = image.convert("RGB").resize((256, 256))
    input_gen = np.array(image_resized).astype(np.float32) / 255.0
    input_gen = input_gen.reshape((1, 256, 256, 3))
    output = generator(input_gen, training=True)[0].numpy()
    output = ((output + 1) / 2 * 255).astype(np.uint8) 
    output_image = Image.fromarray(output)

    return f"{label_text} ({confidence:.2f}%)", output_image

# 隨機載入圖片
def load_random_image():
    folder_path = "./dataset/testedge/"
    images = list(Path(folder_path).glob("*.jpg")) + list(Path(folder_path).glob("*.png"))
    if not images:
        return None

    random_path = random.choice(images)
    image = Image.open(random_path).convert("RGB")  # 灰階

    # 保持比例縮放
    image.thumbnail((256, 256), Image.Resampling.LANCZOS)

    # 建立 256x256 的白底畫布，並將圖片貼上去（置中）
    canvas = Image.new("L", (256, 256), color=255)
    offset = ((256 - image.width) // 2, (256 - image.height) // 2)
    canvas.paste(image, offset)

    image_array = np.array(canvas).astype(np.uint8)

    return {
        "composite": image_array,
        "layers": [image_array],
        "background": None
    }

with gr.Blocks() as demo:
    gr.Markdown("# 🐾 AI Cat/Dog Generator")
    gr.Markdown("選擇上傳圖片或進行手繪，AI 會進行分類與風格生成")

    with gr.Tabs():
        with gr.Tab("📤 上傳圖片"):
            with gr.Row():
                upload_input = gr.Image(
                    type="pil",
                    label="上傳圖片",
                )
            upload_output_label = gr.Textbox(label="分類結果")
            upload_output_image = gr.Image(label="生成圖像")
            upload_submit_btn = gr.Button("✨ 進行分類與生成")
            upload_submit_btn.click(predict_and_generate, inputs=upload_input, outputs=[upload_output_label, upload_output_image])

        with gr.Tab("✏️ 手繪模式"):
            with gr.Row():
                draw_input = gr.Sketchpad(
                    label="手繪或貼上隨機圖片",
                    brush=gr.Brush(colors=["black", "gray"]),
                    canvas_size=(256, 256),
                    # type="numpy"
                )
            draw_output_label = gr.Textbox(label="分類結果")
            draw_output_image = gr.Image(label="生成圖像")
            with gr.Row():
                draw_submit_btn = gr.Button("✨ 進行分類與生成")
                draw_clear_btn = gr.Button("🧹 清除")
                draw_random_btn = gr.Button("🎲 載入隨機圖片")

            draw_submit_btn.click(predict_and_generate, inputs=draw_input, outputs=[draw_output_label, draw_output_image])
            draw_clear_btn.click(lambda: None, None, draw_input, queue=False)
            draw_random_btn.click(load_random_image, outputs=draw_input)

demo.launch(server_name="0.0.0.0", server_port=7860)

/home/handsomeguy/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-18 23:19:21.754532: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-18 23:19:21.795891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758208761.834048   30210 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758208761.846680   30210 cuda_blas.cc:1407] Unabl

開始載入 VGG19 模型...


I0000 00:00:1758208767.950909   30210 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


已載入 VGG19 模型
開始載入 Cat Generator 模型...


已載入 Cat Generator 模型
開始載入 Dog Generator 模型...


已載入 Dog Generator 模型
* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.
